In [1]:
import pandas as pd
import openpyxl
# ram_df = pd.read_excel("./docs/0926_new.xlsx")

In [2]:
loaddf = pd.read_json("vector_all.json")

In [ ]:
loaddf

In [4]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import jieba

In [ ]:
# Generate a word cloud image
chinese_font_path = "jf-openhuninn-2.0.ttf"
def create_word_cloud(index, string):
    wordcloud = WordCloud(
        background_color="white",
        width=600,
        height=400,
        margin=2,
        font_path=chinese_font_path,
        prefer_horizontal=1,
        colormap="Set2",
    ).generate(string)
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    save_path = f"images/{index}.png"
    plt.savefig(save_path, dpi=300, bbox_inches='tight')

In [ ]:
loaddf["jieba_cut"] = loaddf["content"].apply(lambda x: " ".join(jieba.cut(x)))

In [ ]:
loaddf.apply(lambda x: create_word_cloud(x.name, x["jieba_cut"]), axis=1)

In [32]:
for index, row in loaddf.iterrows():
    if (index < 705):
        break
    print("Running:", index)
    stringtext = row["jieba_cut"]
    wordcloud = WordCloud(
        background_color="white",
        width=600,
        height=400,
        margin=2,
        font_path=chinese_font_path,
        prefer_horizontal=1,
        colormap="Set2",
    ).generate(stringtext)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    save_path = f"images/{index}.png"
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    # break

In [ ]:
print("done")

In [5]:
from pymongo import MongoClient

# 建立 MongoDB 連線
client = MongoClient("mongodb://100.90.12.119:27017/")
db = client.nthu_trello_helper
mongo_collection = db.article

---


In [16]:
from pymilvus import DataType, FieldSchema, CollectionSchema, Collection, connections
# 設定 Milvus 的連線資訊
connections.connect(alias="default")

# 設定 Milvus collection 名稱和維度
collection_name = 'trello_finder'
# 設定分區名稱
partition_name = 'import_v5'

In [17]:
milvus_collection = Collection(collection_name)

In [19]:
errorlist = []

---

# Save Data


In [20]:
# 將資料存入 MongoDB
for idx, row in df.iterrows():
    try:
        # 插入 MongoDB
        mongo_collection.insert_one({
            "link_id": str(idx),
            "title": row["title"],
            "url": row["url"],
            "content": row["content"],
            "importMilvus": True,
            "values": row["vector"],
            "class": row["class"],
            "tag": row["tag"],
            "tf": row["TF"],
        })

    except Exception as exp:
        print("MongoDB Insert Fail")
        print(exp)
        errorlist.append(idx)

In [21]:
# 將資料存入 Milvus
for idx, row in df.iterrows():
    try:

        insert_milvus_title = row["title"]
        if (len(insert_milvus_title) > 500):
            insert_milvus_title = insert_milvus_title[:497] + "..."

        try:
            # 插入 Milvus
            status = milvus_collection.insert({
                "id": str(idx),
                "track_id": str(idx),
                "value": row["vector"],
                "title": insert_milvus_title,
            }, partition_name=partition_name)

            if (status.insert_count != 1):
                print("Milvus Insert Fail")
                print(status)
                errorlist.append(idx)
            # else:
                # print("Done \n--------")

        except Exception as exp:
            print("--------------------")
            print(str(idx), "|", len(str(idx)))
            print("Vector |", len(row["vector"]))
            print(insert_milvus_title, "|", len(insert_milvus_title))
            print("--------------------")
            print("Milvus Insert Fail")
            print(exp)
            errorlist.append(idx)

    except Exception as exp:
        print("MongoDB Insert Fail")
        print(exp)
        errorlist.append(idx)

In [ ]:
errorlist

In [ ]:
for idx, row in df.iloc[errorlist].iterrows():
    print(idx)
    insert_milvus_title = row["title"]
    if (len(insert_milvus_title) > 75):
        insert_milvus_title = insert_milvus_title[:71] + "..."
    print(len(insert_milvus_title))
    print(row["title"])

---


In [ ]:
expr = 'id in ["21846"]'
milvus_collection.delete(expr)